<a href="https://colab.research.google.com/github/pidipidi/samsung_planning/blob/main/notebooks/N03_DMP_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic Movement Primitives (DMP)




## Outline

This notebook is an implementation of **Dynamic Movement Primitives (DMPs)** within a `robomimic` simulation environment. The implementation is built on top of the [robomimic tutorial](https://colab.research.google.com/drive/1b62r_km9pP40fKF0cBdpdTO2P_2eIbC6?usp=sharing).

---

### **Objectives**

- Understand the fundamentals of DMPs.
- Integrate DMPs within the `robomimic` environment.
- Visualize demonstrated and generated trajectories.
- Modify the implementation through hands-on exercises.

---

### **Table of Contents**

1. Environment Setup
2. Download Demonstration Data
3. Implement the DMP Algorithm
4. Execution Trajectory (Rollout)
5. Visualize Trajectory

---

**Note**: Please run each cell in order. Some cells are left intentionally editable for hands-on experimentation.

## 1. Environment Setup

In [1]:
WS_DIR = "/content/"
%cd $WS_DIR

# Clone the repo and install the basic requirements
!git clone --branch v0.4 https://github.com/ARISE-Initiative/robomimic
!pip install -e robomimic/ > /dev/null

import sys
import os
sys.path.append('./robomimic/')

/content
Cloning into 'robomimic'...
remote: Enumerating objects: 3546, done.
remote: Counting objects: 100% (1647/1647), done.
remote: Compressing objects: 100% (380/380), done.
remote: Total 3546 (delta 1437), reused 1267 (delta 1267), pack-reused 1899 (from 2)
Receiving objects: 100% (3546/3546), 62.07 MiB | 13.83 MiB/s, done.
Resolving deltas: 100% (2467/2467), done.


In [2]:
# install all system dependencies for mujoco-py
!sudo DEBIAN_FRONTEND=noninteractive apt install curl git libgl1-mesa-dev libgl1-mesa-glx libglew-dev \
         libosmesa6-dev software-properties-common net-tools unzip vim \
         virtualenv wget xpra xserver-xorg-dev libglfw3-dev patchelf > /dev/null

#install mujoco-py
!pip install mujoco > /dev/null

#install robosuite
!pip install robosuite > /dev/null



Extracting templates from packages: 100%


## 2. Download Demonstration Dataset

We download the **robomimic demonstration dataset** for the `"can"` delivery task, which includes proprioceptive data of the robot arm and object state information from expert demonstrations. To train DMPs, we extract the following demonstrated trajectories:

- End-effector position (`robot0_eef_pos`)
- End-effector orientation (`robot0_eef_quat`)
- Object states (`object`)

These demonstrations serve for learning and generating trajectories with DMPs.

**Note:**  
- In Colab, rendering with MuJoCo requires `osmesa` or `egl`.  
- This code sets `MUJOCO_GL` to `osmesa` by default for compatibility.

In [3]:
# import all utility functions
import os
import json
import h5py
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from scipy.spatial.transform import Rotation as R

import robomimic
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.test_utils as TestUtils
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.train_utils as TrainUtils
from robomimic.utils.dataset import SequenceDataset

from robomimic.config import config_factory
from robomimic.algo import algo_factory

# for rendering mujoco in colab, you need turn on egl
os.environ['MUJOCO_GL'] = 'osmesa' # if gpu possible, use 'egl', if not, use 'osmesa'

# the dataset registry can be found at robomimic/__init__.py
from robomimic import DATASET_REGISTRY, HF_REPO_ID

# set download folder and make it
download_folder = WS_DIR + "robomimic_data/"
os.makedirs(download_folder, exist_ok=True)

# download the dataset
task = "can"
dataset_type = "ph"
hdf5_type = "low_dim"
FileUtils.download_file_from_hf(
    repo_id=HF_REPO_ID,
    filename=DATASET_REGISTRY[task][dataset_type][hdf5_type]["url"],
    download_dir=download_folder,
)

# enforce that the dataset exists
dataset_path = os.path.join(download_folder, "low_dim_v15.hdf5")
assert os.path.exists(dataset_path)

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /content/./robomimic/robomimic/scripts/setup_macros.py
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


low_dim_v15.hdf5:   0%|          | 0.00/46.9M [00:00<?, ?B/s]

To confirm the downloaded demonstrations, we playback five demonstrations recording a video file:

In [4]:
!python robomimic/robomimic/scripts/playback_dataset.py --dataset robomimic_data/low_dim_v15.hdf5 --video_path demo_playback_dataset.mp4 --video_skip 1 --n 5

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /content/robomimic/robomimic/scripts/setup_macros.py
)

============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_eef_pos']
using obs modality: rgb with keys: []
[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /usr/local/lib/python3.11/dist-packages/robosuite/scripts/setup_macros.py (macros.py:55)
[robosuite WARNING] Could not import robosuite_models. Some robots may not be available. If you want to use these robots, please install robosuite_models from source (https://github.com/ARISE-Initiative/robosuite_models) or through pip install. (__init__.py:30)
[robosuite WARNING] Could not load the mink-based whole-body IK. Make sure you install related import properl

In [5]:
# visualize rollout video

from IPython.display import HTML
from base64 import b64encode

mp4 = open("demo_playback_dataset.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=400 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")

Based on the above playback visualization, we split the behvior into three primitives: reaching, grasping, and placing. To learn each primitive, we extract necessary information from the demonstrations. The extraction process consists of the following steps:

1. We first extract end-effector position (`eef_pos`), orientation (`quat`), and object state (`obj`) information.
2. We then identify each primitive phase:
  - **reach phase**: the gripper approaches to an object.
  - **grasp phase**: the gripper moves away after grasping.
  - **finish phase**: the gripper releases the object in hand.  
3. We store positions and orientations at key frames such as:
  - `ee_to_obj_pos` – the relative position of the object in the end-effector frame before grasping.
  - `grasp_ee_pos` – the end-effector position at grasping moment.
  - `finish_ee_pos` – the end-effector position at episode end.


In [6]:
def decided_phase(data):
    gripper_acc = np.diff(np.diff(data['obs/robot0_gripper_qpos']), axis=0)

    reach_end = np.argmax(gripper_acc > 5e-4)
    grasp_end = reach_end + np.argmax(gripper_acc[reach_end:] < -5e-4)

    return reach_end, grasp_end

f = h5py.File(dataset_path, "r")
dataset = f["data"]

reach_dataset = []
grasp_dataset = []
finish_dataset = []
keyframe_infos = []
for i in dataset.keys():
    eef_pos = np.array(dataset[i]['obs/robot0_eef_pos']) # in robot base frame
    quat = np.array(dataset[i]['obs/robot0_eef_quat']) # in robot base frame
    obj = np.array(dataset[i]['obs/object'])
    reach_end, grasp_end = decided_phase(dataset[i])

    # Split demonstrated trajectories
    reach_dataset.append({'eef_pos': eef_pos[:reach_end], 'quat': quat[:reach_end], 'obj': obj[:reach_end]})
    grasp_dataset.append({'eef_pos': eef_pos[reach_end:grasp_end], 'quat': quat[reach_end:grasp_end], 'obj': obj[reach_end:grasp_end]})
    finish_dataset.append({'eef_pos': eef_pos[grasp_end:], 'quat': quat[grasp_end:], 'obj': obj[grasp_end:]})

    # Store key frames
    keyframe_infos.append({
        'ee_to_obj_pos': obj[reach_end-1, :3], # in ee frame
        'ee_to_obj_quat': obj[reach_end-1, 3:7], # in ee frame
        'grasp_ee_pos': eef_pos[grasp_end-1],
        'grasp_ee_quat': quat[grasp_end-1],
        'finish_ee_pos': eef_pos[-1],
        'finish_ee_quat': quat[-1],
    })

ee_to_obj_pos = np.mean(np.stack([d['ee_to_obj_pos'] for d in keyframe_infos]), axis=0)
ee_to_obj_quat = np.mean(np.stack([d['ee_to_obj_quat'] for d in keyframe_infos]), axis=0)
grasp_ee_pos = np.mean(np.stack([d['grasp_ee_pos'] for d in keyframe_infos]), axis=0)
grasp_ee_quat = np.mean(np.stack([d['grasp_ee_quat'] for d in keyframe_infos]), axis=0)
finish_ee_pos = np.mean(np.stack([d['finish_ee_pos'] for d in keyframe_infos]), axis=0)
finish_ee_quat = np.mean(np.stack([d['finish_ee_quat'] for d in keyframe_infos]), axis=0)


## 3. Implement DMP Algorithm



We now implement the **Dynamic Movement Primitives (DMP)** framework to generate smooth, goal-directed trajectories from demonstrations.

### Key Concepts

* **Canonical System**:

  Acts as a phase variable that monotonically decays from 1 to 0 (for discrete movements).
  
  Governs temporal progression:
  $\tau\dot{s} = -\alpha s$
  
  This variable controls how basis functions are activated over time.

* **Transformation System**:

  Models a damped spring system augmented with a learnable forcing term:

  $$
  \tau \dot{v}=K(g-x)-Dv+(g-x_0)f
  $$
  
  $$
  \tau \dot{x}=v
  $$

  * \$x, v\$: current position and velocity
  * \$x_0, g\$: start and goal position
  * \$\tau\$: temporal scaling factor
  * \$K\$: acts like spring constant
  * \$D\$: damping term
  * \$f\$: forcing term

* **Forcing Term \$f(s)\$**:

  A weighted sum of radial basis functions (RBFs), modulated by the canonical variable:

  $$
  f(s) = \frac{\sum_i w_i\psi_i(s) s}{\sum_i \psi_i(s)}
  $$

  where each basis function \$\psi\_i(s) = \exp(-h\_i (s - c\_i)^2)\$.

* **Weight Learning**:

  From demonstrated trajectories, compute target acceleration and solve for weights \${w\_i}\$ using weighted linear regression.

### Rollout Execution

After weight learning, the system simulates a trajectory with:

1. Phase variable update.
2. Basis function activation (`ψ_i(s)`).
3. Compute forcing term \$f(s)\$.
4. Integrate acceleration → velocity → position.

---

> ✅ The DMP system allows for **modulating**, **rescaling**, and **generalizing** trajectories with respect to new start/goal or execution speeds.

In [7]:
import matplotlib.pylab as plt
class TransformationSystem(object):
    """ This system models a damped spring system augmented with a learnable forcing term """

    def __init__(self, K=50., D=None):
        """
        K float: spring constant
        D float: damping constant
        """
        self.K = K
        if D is None:
            D = 2.0 * np.sqrt(self.K)
        self.D = D

    def acceleration(self, x, dx, start, goal, tau, f, s):
        """
        tau float: scaling parameter for execution speed
        f float: forcing term
        """
        return (self.K*(goal-x) - self.D*dx + (goal-start)*f) / tau

    def fs(self, x, dx, ddx, start, goal, tau, s):
        """
        tau float: scaling parameter for execution speed
        f float: forcing term
        """
        return (tau*ddx - self.K*(goal-x) + self.D*dx) / (goal-start)


class DMPs_discrete(object):

    def __init__(self, dims, bfs, dt=.01, tau=1., alpha=14, enable_improved=False, **kwargs):
        '''
        dims int: number of dynamic motor primitives
        bfs int: number of basis functions per DMP
        dt float: timestep for simulation
        tau float: scales the timestep
                   increase tau to make the system execute faster
        alpha float: canonical system parameter
        '''

        self.dmps = dims
        self.bfs  = bfs
        self.dt   = dt
        self.tau  = tau
        self.alpha = alpha/2.0
        self.alpha_x = alpha

        self.prep_centers_and_variances()
        self.formulation = TransformationSystem()
        return


    def prep_centers_and_variances(self):
        '''
        Set the centre of the Gaussian basis functions be spaced evenly
        throughout run time.
        '''
        self.c = np.zeros(self.bfs)
        self.h = np.zeros(self.bfs)

        t = np.linspace(0,1,self.bfs) *0.5

        # From DMP matlab code
        self.c = np.exp(-self.alpha_x*t)
        self.D = (np.diff(self.c)*0.55)**2
        self.D = np.append(self.D, self.D[-1])
        self.D = 1/self.D
        self.h = np.ones(self.bfs)*0.5


    def gen_psi(self, x):
        '''
        Generates the activity of the basis functions for a given state of the
        canonical system.

        x float: the current state of the canonical system
        '''
        if isinstance(x, np.ndarray):
            x = x[:,None]
        return np.exp(-self.h * (x - self.c)**2 * self.D)


    def gen_phase(self, n_steps, tau=None):
        """
        Generate phase for open loop movements.

        n_steps int: number of steps
        """
        if tau is None: tau = self.tau
        return np.exp(-self.alpha/tau * np.linspace(0, 1, n_steps))


    def learn(self, y_des):
        """
        Encode a set of weights from the input trajectories.

        y_des list/array: the desired trajectories of each DMP
                          should be shaped [dmps, run_time]
        """
        # Set variables
        n_samples, dims, n_steps = np.shape(y_des)
        self.n_steps = n_steps
        assert dims==self.dmps, "wrong dimensions"

        # Get start and goal
        self.y0   = np.mean(y_des[:,:,0], axis=0)
        self.goal = np.mean(y_des[:,:,-1], axis=0)

        # Calculate yd_des, ydd_des
        yd_des = np.diff(y_des) / self.dt
        yd_des = np.concatenate((np.zeros((n_samples, self.dmps, 1)), yd_des), axis=2)

        ydd_des = np.diff(yd_des) / self.dt
        ydd_des = np.concatenate((np.zeros((n_samples, self.dmps, 1)), ydd_des), axis=2)

        # Get a canonical system
        x_track = self.gen_phase(n_steps)

        # Calculate f
        f_des = np.zeros((n_samples, self.dmps, n_steps))
        for i in range(n_samples):
            for j in range(self.dmps):
                f_des[i,j] = self.formulation.fs(y_des[i,j], yd_des[i,j], ydd_des[i,j],
                                                     self.y0[j:j+1], self.goal[j:j+1], self.tau,
                                                     x_track)

        # Calculate weights
        psi_track = self.gen_psi(x_track)

        x_track   = np.tile(x_track, n_samples)
        psi_track = np.tile(psi_track, (n_samples,1))
        f_des     = np.swapaxes(f_des, 0, 1)
        f_des     = f_des.reshape((self.dmps,n_samples*n_steps))

        self.w = np.zeros((self.dmps, self.bfs))
        for d in range(self.dmps):
            for b in range(self.bfs):
                numer = np.sum(x_track    * psi_track[:,b] * f_des[d,:])
                denom = np.sum(x_track**2 * psi_track[:,b]) + 1e-10
                self.w[d,b] = numer / denom

        # set up tracking vectors
        y_track   = np.zeros((self.dmps, n_steps))
        yd_track  = np.zeros((self.dmps, n_steps))
        ydd_track = np.zeros((self.dmps, n_steps))

        y   = self.y0.copy()
        yd  = np.zeros(self.dmps)
        ydd = np.zeros(self.dmps)

        x_track   = self.gen_phase(n_steps)
        psi_track = self.gen_psi(x_track)

        f = np.zeros((self.dmps,n_steps))
        for idx in range(self.dmps):
            f[idx] = np.sum(np.expand_dims(x_track,axis=1)*np.expand_dims(self.w[idx],axis=0)*psi_track, axis=1)/ np.sum(psi_track, axis=1)

        # Recover the demonstration using the learned weights (for confirmation)
        for t in range(n_steps):
            for idx in range(self.dmps):
                # Calcualte acceleration based on f(s)
                ydd[idx] = self.formulation.acceleration(y[idx], yd[idx], self.y0[idx],
                                                             self.goal[idx], self.tau,
                                                            f[idx,t], x_track[t])
                yd[idx] += ydd[idx] * self.dt * self.tau
                y[idx]  += yd[idx] * self.dt

            # record timestep
            y_track[:,t] = y
            yd_track[:,t] = yd
            ydd_track[:,t] = ydd

        return y_track, yd_track, ydd_track


    def plan(self, y0=None, goal=None, **kwargs):
        '''
        Run the DMP system within a specific period.

        y0   list/array: start position
        goal list/array: goal position
        tau  float:      scales the timestep
                         increase tau to make the system execute faster
        '''

        if y0 is None: y0 = self.y0
        if goal is None: goal = self.goal
        n_steps = int(self.n_steps/self.tau)

        # set up tracking vectors
        y_track   = np.zeros((self.dmps, n_steps))
        yd_track  = np.zeros((self.dmps, n_steps))
        ydd_track = np.zeros((self.dmps, n_steps))
        x_track   = self.gen_phase(n_steps, self.tau)

        y   = y0.copy()
        yd  = np.zeros(self.dmps)
        ydd = np.zeros(self.dmps)

        for t in range(n_steps):
            for idx in range(self.dmps):
                # Calcualte acceleration based on f(s)
                psi = self.gen_psi(x_track[t])
                f   = x_track[t] * ((np.dot(psi, self.w[idx])) / np.sum(psi))

                ydd[idx] = self.formulation.acceleration(y[idx], yd[idx], y0[idx], goal[idx], self.tau,
                                                        f, x_track[t])
                yd[idx] += ydd[idx] * self.tau * self.dt
                y[idx]  += yd[idx] * self.dt

            # record timestep
            y_track[:,t] = y
            yd_track[:,t] = yd
            ydd_track[:,t] = ydd

        return y_track, yd_track, ydd_track


    def plot_traj(self, trajs_demo, trajs_gen, axis_num=0):
        """Plot trajectories over an axis """

        fig = plt.figure()
        plt.title('Trajectory (X) - Demo (Td) and generated (Tg)')
        for i in range(len(trajs_demo)):
            plt.plot(trajs_demo[i,axis_num,:], 'r--', label='Td')
        for i in range(len(trajs_gen)):
            plt.plot(trajs_gen[i,axis_num,:],'g-', label='Tg')

        plt.legend()
        plt.show()

Then, we define DMP Utilities
- `filtering(trajectories, quat_normalization)`  
  → Smoothing the raw trajectories with zero padding and quaternion normalization (optional).
- `learn_dmp_from_traj(trajectories, bfs)`  
  → Learns a DMP over 7D trajectories (dataset) with a number of basis functions (bfs).
- `make_goal_from_obj(obj_pos, obj_quat, rel_pos, rel_quat)`  
  → Computes an end-effector **goal pose** from an object pose and a relative grasp pose (optional).

In [8]:
from scipy.spatial.transform import RigidTransform as Tf
from scipy import signal

def filtering(traj, quat_normalization=False):
    """ Filter out an multi-dimensional dem trajectory """
    # zero padding
    pad_length = 20
    traj = np.pad(traj, ((pad_length,pad_length),(0,0)), mode='edge')

    # raw x,y
    x = np.linspace(0, 1, len(traj))
    y = np.array(traj)

    # failtered x,y
    b, a = signal.butter(8, 0.03)

    ynew = []
    for i in range(len(y[0])):
        ynew.append( signal.filtfilt(b,a,y[:,i], padtype='constant'))

    ynew = np.array(ynew).T
    xnew = np.linspace(0, 1.0, len(ynew))

    if quat_normalization:
        print("Normalize quaternions")
        ynew[:,3:] = ynew[:,3:]/np.expand_dims(np.linalg.norm(ynew[:,3:], axis=1), axis=1)

    return xnew, ynew

def learn_dmp_from_traj(dataset, bfs=10):
    """ train DMPs from (single) demonstration dataset """

    trajs = [np.concatenate([dataset[0]['eef_pos'], dataset[0]['quat']], axis=-1)]

    # filtering and zero padding
    trajs = [filtering(traj)[1] for traj in trajs]

    trajs = np.swapaxes(trajs, 1, 2)
    D, _ = trajs[0].shape
    dmp = DMPs_discrete(dims=D, bfs=bfs, tau=1)
    dmp.learn(trajs)

    #temp
    #traj, _, _ = dmp.plan(y0=dmp.y0, goal=dmp.goal)
    #dmp.plot_traj(trajs, np.expand_dims(traj, axis=0), axis_num=2)
    return dmp

def make_goal_from_obj(obj_pos, obj_quat, rel_pos=None, rel_quat=None):
    """ Computes an end-effector goal pose from an object pose and a relative grasp pose"""

    if rel_pos is None:
      rel_pos = np.zeros(3)
    if rel_quat is None:
      rel_quat = np.array([0,0,0,1])

    T_ee_to_obj   = Tf.from_components(rel_pos, R.from_quat(rel_quat))
    T_base_to_obj = Tf.from_components(obj_pos, R.from_quat(obj_quat))
    T_base_to_ee  = T_base_to_obj * T_ee_to_obj.inv()
    tf_t, tf_r = T_base_to_ee.as_components()

    return np.concatenate([tf_t, tf_r.as_quat()])

Here, we make segment-wise DMPs from demonstrations.

- Learn DMPs for each motion phase:
  - `reach`: from start to contact
  - `grasp`: close gripper and transport
  - `finish`: release


In [9]:
reach_dmp = learn_dmp_from_traj(reach_dataset, bfs=30)
grasp_dmp = learn_dmp_from_traj(grasp_dataset, bfs=30)
finish_dmp = learn_dmp_from_traj(finish_dataset, bfs=5)

## 4. Path Execution (Rollout)

Now that we've implemented DMPs, we will **execute actions in the robomimic**.

### Environment Setup

We first create a robomimic-compatible environment from metadata in the dataset.  
This ensures that the simulation settings (robot type, object type, etc.) exactly match the demonstration data.
Please, ignore warning messages.

In [10]:
import robomimic.utils.env_utils as EnvUtils

dummy_spec = dict(
    obs=dict(
        low_dim=["robot0_eef_pos", "robot0_eef_quat", "robot0_gripper_qpos", "object"],
        rgb=[],
    ),
)
ObsUtils.initialize_obs_utils_with_obs_specs(obs_modality_specs=dummy_spec)

env_meta = FileUtils.get_env_metadata_from_dataset(dataset_path)

env = EnvUtils.create_env_from_metadata(
    env_meta=env_meta,
    env_name=env_meta["env_name"],
    render=False,
    render_offscreen=True,
    use_image_obs=False,
)

[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /usr/local/lib/python3.11/dist-packages/robosuite/scripts/setup_macros.py (macros.py:55)



============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_eef_quat', 'robot0_gripper_qpos', 'object', 'robot0_eef_pos']
using obs modality: rgb with keys: []


[robosuite WARNING] Could not import robosuite_models. Some robots may not be available. If you want to use these robots, please install robosuite_models from source (https://github.com/ARISE-Initiative/robosuite_models) or through pip install. (__init__.py:30)
[robosuite WARNING] Could not load the mink-based whole-body IK. Make sure you install related import properly, otherwise you will not be able to use the default IK controller setting for GR1 robot. (__init__.py:40)


Created environment with name PickPlaceCan
Action size is 7


### Rollout Function

We define a `rollout` function that:
- Takes in the environment and a trajectory
- Compute a desired action command based on the current state at each step
- Executes the command in the simulation, one step at a time
- Records frames for visualization
- Tracks success metrics such as task completion and total reward


In [11]:
def rollout(env, traj, video_writer):
    results = {}

    video_frames = []
    video_count = 0  # video frame counter

    rews = []
    success = None # success metrics

    for s in traj:

        # design a simple feedback controller to compute an action command
        obs = env.get_observation()
        cur_pos  = obs['robot0_eef_pos']
        cur_quat = obs['robot0_eef_quat']
        vel_cmd  = 80.0*(s[:3]-obs['robot0_eef_pos']) # 80.0 is a position gain
        rot_cmd  = 3.0*(R.from_quat(s[3:7]) * R.from_quat(cur_quat).inv()).as_rotvec() # 5.0 is a rotation gain
        cmd = np.concatenate([vel_cmd, rot_cmd, s[-1:]], axis=-1)

        # play action
        ob_dict, r, done, _ = env.step(cmd)

        # compute reward
        rews.append(r)

        cur_success_metrics = env.is_success()

        if success is None:
            success = deepcopy(cur_success_metrics)
        else:
            for k in success:
                success[k] = success[k] | cur_success_metrics[k]

        # visualization
        frame = env.render(mode="rgb_array", height=512, width=512)
        video_frames.append(frame)
        video_count += 1

        # break if done
        if done:
            end_step = video_count
            break

    for frame in video_frames:
        video_writer.append_data(frame)

    end_step = video_count
    total_reward = np.sum(rews[:end_step + 1])

    results["Return"] = total_reward
    results["Horizon"] = end_step + 1
    results["Success_Rate"] = float(success["task"])

    # log additional success metrics
    for k in success:
        if k != "task":
            results["{}_Success_Rate".format(k)] = float(success[k])

    return ob_dict, results

###Action Execution Steps

We execute **three DMP rollouts**, corresponding to stages in a typical manipulation task:

1. **Reach the Object**
   - Plan a path from the robot's current end-effector position to the object.
   - Use DMP to generate the trajectory and convert it to a list of actions.
   - Execute the actions with gripper open (`-1`).

2. **Grasp and Transport**
   - Plan a new path from the grasp point to a transport goal position.
   - Execute with gripper closed (`1`).

3. **Arrive and Release**
   - Execute with gripper open (`-1`) to release.
   
   ---

> 📌 Note:  
> All actions are formatted as `[x, y, z, rx, ry, rz, gripper]` where `gripper` is `-1` (open) or `1` (close).


In [12]:
def rollout_phase(phase, dmp, ob_dict, video_writer):
    y0 = np.concatenate([ob_dict["robot0_eef_pos"], ob_dict["robot0_eef_quat"]], axis=-1)
    gripper = -1
    if phase == "reach":
        goal = make_goal_from_obj(ob_dict["object"][7:10], ob_dict["object"][10:], ee_to_obj_pos, ee_to_obj_quat)
    elif phase == "grasp":
        goal = make_goal_from_obj(grasp_ee_pos, grasp_ee_quat)
        gripper = 1
    elif phase == "finish":
        goal = make_goal_from_obj(finish_ee_pos, finish_ee_quat)

    # Generate a desired trajectory
    traj, _, _ = dmp.plan(y0=y0, goal=goal)
    traj = np.swapaxes(traj, 0, 1)
    T, _ = traj.shape

    # Attach a gripper command trajectory
    gripper_traj = np.full((T, 1), gripper)
    traj = np.concatenate([traj, gripper_traj], axis=-1)

    ob_dict, results = rollout(env, traj, video_writer)
    return ob_dict, results

In [13]:
from robomimic.algo import RolloutPolicy
import imageio
from copy import deepcopy

# create a video writer
video_path = "rollout.mp4"
video_writer = imageio.get_writer(video_path, fps=20)

env.reset()
ob_dict, _, _, _ = env.step(np.zeros(7))

ob_dict, results = rollout_phase("reach", reach_dmp, ob_dict, video_writer)
ob_dict, results = rollout_phase("grasp", grasp_dmp, ob_dict, video_writer)
ob_dict, results = rollout_phase("finish", finish_dmp, ob_dict, video_writer)

video_writer.close()

## 5. Path Visualization

### Video Logging

- A video of the entire trajectory is saved as `rollout.mp4`.

In [14]:
# visualize rollout video
from IPython.display import HTML
from base64 import b64encode

mp4 = open(video_path, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=400 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")